In [ ]:
from dataclasses import dataclass
from pathlib import Path

from aoc.strings import split_in_to_characters

data_file = Path("../Data/day6.txt").read_text()

EXAMPLE = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""

OBSTACLE = "#"
SELF_SET_OBSTACLE = "O"


@dataclass
class Cell:
    row: int
    column: int
    value: str

    @property
    def hash_value(self):
        return (self.row, self.column)


class GuardMap:
    items: list[list[Cell]]
    traveled: set[tuple[int, int]]

    def __init__(self, items: list[list[Cell]]) -> None:
        self.items = items
        self.traveled = set()

    def print_map(self):
        for row in self.items:
            print("".join(map(lambda x: x.value, row)))

    def set_cell(self, cell: Cell | tuple[int, int], value: str):
        if isinstance(cell, Cell):
            self.items[cell.row][cell.column].value = value
        else:
            self.items[cell[0]][cell[1]].value = value

    def count_traveled(self):
        return len(self.traveled)

    def move_forward(self, start: Cell, direction: str):
        self.__mark_traveled(start)
        moves = {
            "^": self.__move_up,
            ">": self.__move_right,
            "v": self.__move_down,
            "<": self.__move_left,
        }

        return moves[direction](start)

    def find_character(self, search: str):
        for row_line in self.items:
            for cell in row_line:
                if cell.value == search:
                    return cell

        raise Exception("Nothing to be found here!")

    def __mark_traveled(self, cell: Cell):
        self.traveled.add((cell.row, cell.column))
        self.set_cell(cell, "X")

    def __hit_obstacle(self, cell: Cell):
        return cell.value == OBSTACLE or cell.value == SELF_SET_OBSTACLE

    def __move_up(self, start: Cell):
        last_cell: Cell | None = None
        hit_wall = True
        for row in reversed(range(0, start.row)):
            cell = self.items[row][start.column]
            if self.__hit_obstacle(cell):
                hit_wall = False
                break

            self.__mark_traveled(cell)
            last_cell = cell

        return self.__end_moving(last_cell, hit_wall)

    def __move_down(self, start: Cell):
        last_cell: Cell | None = None
        hit_wall = True
        for row in range(start.row, len(self.items)):
            cell = self.items[row][start.column]
            if self.__hit_obstacle(cell):
                hit_wall = False
                break

            self.__mark_traveled(cell)
            last_cell = cell

        return self.__end_moving(last_cell, hit_wall)

    def __move_right(self, start: Cell):
        last_cell: Cell | None = None
        hit_wall = True
        for column in range(start.column, len(self.items[start.row])):
            cell = self.items[start.row][column]
            if self.__hit_obstacle(cell):
                hit_wall = False
                break

            self.__mark_traveled(cell)
            last_cell = cell

        return self.__end_moving(last_cell, hit_wall)

    def __move_left(self, start: Cell):
        last_cell: Cell | None = None
        hit_wall = True
        for column in reversed(range(0, start.column)):
            cell = self.items[start.row][column]
            if self.__hit_obstacle(cell):
                hit_wall = False
                break

            self.__mark_traveled(cell)
            last_cell = cell

        return self.__end_moving(last_cell, hit_wall)

    def __end_moving(self, last_cell: Cell | None, hit_wall: bool):
        if not last_cell:
            assert not hit_wall

            return None, False

        return self.items[last_cell.row][last_cell.column], hit_wall


def turn_right(direction: str):
    turns = {"^": ">", ">": "v", "v": "<", "<": "^"}

    return turns[direction]


def prepare(input: str):
    def map_row_line(args: tuple[int, list[str]]):
        row, row_line = args
        cells: list[Cell] = []
        for index, value in enumerate(row_line):
            cells.append(Cell(row=row, column=index, value=value))

        return cells

    guard_map = GuardMap(
        list(
            map(
                map_row_line, enumerate(map(split_in_to_characters, input.splitlines()))
            )
        )
    )

    return guard_map

In [ ]:
def part1(input: str):
    guard_map = prepare(input)
    point = guard_map.find_character("^")
    assert point.value == "^"

    direction = point.value
    should_stop = False
    while not should_stop:
        assert isinstance(point, Cell)
        point, should_stop = guard_map.move_forward(point, direction)
        if should_stop:
            guard_map.set_cell(point, direction)
            break

        direction = turn_right(direction)

    return guard_map.count_traveled()


example_result = part1(EXAMPLE)

print("example result is", example_result)

assert example_result == 41

result = part1(data_file)

print("result is", result)

assert result < 5126
assert result < 5125
assert result > 5003
assert result != 5004
assert result != 5085
assert result == 5086

example result is 41
result is 5086


In [3]:
DIRECTIONS = ["^", "v", ">", "<"]


def part2(input: str):
    guard_map = prepare(input)
    point = guard_map.find_character("^")
    assert point.value == "^"

    starting_point = (point.row, point.column)
    loops: list[Cell] = []
    for row in range(len(guard_map.items)):
        for cell in guard_map.items[row]:
            if cell.value == OBSTACLE or cell.value == "^":
                continue

            assert cell.value not in DIRECTIONS

            # print(f"checking {cell=} if it causes loops")
            guard_map.set_cell(cell, SELF_SET_OBSTACLE)
            direction = point.value
            guard_escaped = False
            ends: set[str] = set()
            while not guard_escaped:
                point, guard_escaped = guard_map.move_forward(point, direction)
                if guard_escaped or point is None:
                    break

                end_value = f"({point.row}, {point.column}, {direction})"
                if end_value in ends:
                    loops.append(cell)
                    break

                ends.add(end_value)
                direction = turn_right(direction)

            # Reset to initial values
            guard_map.set_cell(starting_point, "^")
            guard_map.set_cell(cell, ".")
            point = guard_map.items[starting_point[0]][starting_point[1]]

        print(f"analyzed {row=} and found {len(loops)=}")

    return len(loops)


example_result = part2(EXAMPLE)

print("example result is", example_result)

assert example_result == 6

result = part2(data_file)

print("result is", result)

assert result > 1624

analyzed row=0 and found len(loops)=0
analyzed row=1 and found len(loops)=0
analyzed row=2 and found len(loops)=0
analyzed row=3 and found len(loops)=0
analyzed row=4 and found len(loops)=0
analyzed row=5 and found len(loops)=0
analyzed row=6 and found len(loops)=1
analyzed row=7 and found len(loops)=3
analyzed row=8 and found len(loops)=5
analyzed row=9 and found len(loops)=6
example result is 6
analyzed row=0 and found len(loops)=0
analyzed row=1 and found len(loops)=0
analyzed row=2 and found len(loops)=0
analyzed row=3 and found len(loops)=0
analyzed row=4 and found len(loops)=0
analyzed row=5 and found len(loops)=41
analyzed row=6 and found len(loops)=62
analyzed row=7 and found len(loops)=62
analyzed row=8 and found len(loops)=63
analyzed row=9 and found len(loops)=89
analyzed row=10 and found len(loops)=92
analyzed row=11 and found len(loops)=106
analyzed row=12 and found len(loops)=121
analyzed row=13 and found len(loops)=122
analyzed row=14 and found len(loops)=144
analyzed ro

AssertionError: 